In [214]:
import os

import pandas as pd
import numpy as np


%run us_common_functions.ipynb



combine_studies = False


In [215]:
single_sec_df = pd.read_csv(all_studies_data_dir + 'data_2.5.csv')
window_sec_df = pd.read_csv(all_studies_data_dir + 'data_3.csv')

single_sec_df.head(5)
window_sec_df.head(5)

,Subject,Age,NASA_Effort,NASA_Frustration,NASA_Mental,NASA_Performance,NASA_Physical,NASA_Temporal,NASA_Total,Gender,Gender_Male,Perinasal_Mean_NN,Perinasal_Median_NN,Perinasal_SD_NN,Perinasal_SS_NN,Hr_Mean_NN,Hr_Median_NN,Hr_SD_NN,Hr_SS_NN,Br_Mean_NN,Br_Median_NN,Br_SD_NN,Br_SS_NN,Perinasal_Mean,Perinasal_Median,Perinasal_SD,Perinasal_SS,HR_Mean,HR_Median,HR_SD,HR_SS,BR_Mean,BR_Median,BR_SD,BR_SS,PP_Arousal_Mode,HR_Arousal_Mode,BR_Arousal_Mode,PP_HR_Arousal_Mode,PP_BR_Arousal_Mode,HR_BR_Arousal_Mode,PP_HR_BR_Arousal_Mode,Study_Name,Study_Subject,Treatment,PP_RB_Mean,HR_RB_Mean,BR_RB_Mean
0,2,23.0,17.0,11.0,18.0,8.0,12.0,11.0,77.0,1,0,-5.234157,-5.225352,0.022537,273.968589,90.34,90.3,1.484887,81633.00,11.052,11.05,0.162809,1221.7056,0.020212,0.029017,0.022537,0.008657,3.708182,3.668182,1.484887,157.350124,-5.414682,-5.416682,0.162809,293.426352,neutral,stressed,relaxed,neutral,relaxed,neutral,neutral,sim1,sim1_2,RD,-5.254369,86.631818,16.466682
1,2,23.0,17.0,11.0,18.0,8.0,12.0,11.0,77.0,1,0,-5.310385,-5.314152,0.010595,282.002952,91.20,90.6,1.366260,83191.20,12.004,12.10,0.352742,1442.0800,-0.056016,-0.059782,0.010595,0.032388,4.568182,3.968182,1.366260,225.482851,-4.462682,-4.366682,0.352742,200.275130,relaxed,stressed,relaxed,neutral,relaxed,neutral,neutral,sim1,sim1_2,RD,-5.254369,86.631818,16.466682
2,2,23.0,17.0,11.0,18.0,8.0,12.0,11.0,77.0,1,0,-5.301983,-5.300184,0.007855,281.110844,90.08,90.3,1.347261,81160.40,13.108,13.10,0.355928,1719.3368,-0.047614,-0.045814,0.007855,0.023226,3.448182,3.668182,1.347261,135.235579,-3.358682,-3.366682,0.355928,113.947596,relaxed,stressed,relaxed,neutral,relaxed,neutral,neutral,sim1,sim1_2,RD,-5.254369,86.631818,16.466682
3,2,23.0,17.0,11.0,18.0,8.0,12.0,11.0,77.0,1,0,-5.376241,-5.384108,0.022728,289.044320,86.64,86.8,1.188089,75077.60,14.164,14.08,0.319138,2007.1056,-0.121872,-0.129739,0.022728,0.153176,0.008182,0.168182,1.188089,12.704669,-2.302682,-2.386682,0.319138,53.940076,relaxed,neutral,relaxed,relaxed,relaxed,neutral,relaxed,sim1,sim1_2,RD,-5.254369,86.631818,16.466682
4,2,23.0,17.0,11.0,18.0,8.0,12.0,11.0,77.0,1,0,-5.412159,-5.415029,0.008453,292.915248,86.70,87.6,1.708150,75195.16,15.488,15.55,0.339830,2399.8208,-0.157789,-0.160660,0.008453,0.249618,0.068182,0.968182,1.708150,26.306488,-0.978682,-0.916682,0.339830,10.617541,relaxed,neutral,neutral,neutral,relaxed,neutral,neutral,sim1,sim1_2,RD,-5.254369,86.631818,16.466682


In [216]:
single_sec_df.columns

Index(['Subject', 'Age', 'NASA_Effort', 'NASA_Frustration', 'NASA_Mental',
       'NASA_Performance', 'NASA_Physical', 'NASA_Temporal', 'NASA_Total',
       'TimeElapsed', 'Treatment', 'Gender', 'Perinasal', 'Perinasal_Log',
       'Heart', 'Breathing', 'PP_Normalized', 'HR_Normalized', 'BR_Normalized',
       'PP_Arousal', 'HR_Arousal', 'BR_Arousal', 'PP_ecdf', 'HR_ecdf',
       'BR_ecdf', 'PP_HR_ecdf', 'PP_BR_ecdf', 'HR_BR_ecdf', 'PP_HR_BR_ecdf',
       'PP_Arousal_ecdf', 'HR_Arousal_ecdf', 'BR_Arousal_ecdf',
       'PP_HR_Arousal_ecdf', 'PP_BR_Arousal_ecdf', 'HR_BR_Arousal_ecdf',
       'PP_HR_BR_Arousal_ecdf', 'Study_Name', 'Study_Subject'],
      dtype='object')

In [217]:
single_sec_df.Study_Name.unique()

array(['sim1', 'sim2', 'tt1', 'office_tasks', 'deadline_study'],
      dtype=object)

In [218]:
study_ecdf_combinations

{'sim1___sim2': ['sim1', 'sim2'],
 'sim1___sim2___tt1': ['sim1', 'sim2', 'tt1'],
 'sim1___sim2___office_tasks': ['sim1', 'sim2', 'office_tasks'],
 'sim1___sim2___tt1___office_tasks': ['sim1', 'sim2', 'tt1', 'office_tasks']}

In [219]:
study_combined_feaures = ['Subject',
                          'Study_Name',
                          'Study_Subject',
                          'Treatment', 
                          'TimeElapsed',

                          'Gender', 
                          'Age',

                          'NASA_Effort',
                          'NASA_Frustration', 
                          'NASA_Mental', 
                          'NASA_Performance', 
                          'NASA_Physical',
                          'NASA_Temporal', 
                          'NASA_Total', 

                          'Perinasal_Log', 
                          'Heart', 
                          'Breathing',

                          'PP_Normalized', 
                          'HR_Normalized', 
                          'BR_Normalized',

                          'PP_Arousal', 
                          'HR_Arousal',
                          'BR_Arousal'
                         ]

In [220]:
if combine_studies:
    start_time = time.time()

    for study_comb in list(study_ecdf_combinations.keys()):
    # for study_comb in list(study_ecdf_combinations.keys())[1:2]:

        studies = study_ecdf_combinations[study_comb]
        study_name_combined = '___'.join(studies)
        print('\n\n%s\n' % study_name_combined)

        study_comb_signals = ['PP', 'HR', 'BR']
        study_combined_df = single_sec_df.copy()[single_sec_df.Study_Name.isin(studies)]
        study_combined_df = study_combined_df.loc[:, study_combined_feaures]
        # study_combined_df = study_combined_df.loc[:1000, study_combined_feaures]


        for i, label in enumerate(('A', 'B', 'C')):
            study_combined_df = generate_ecdf_plots(study_combined_df, i)

        study_combined_df['Subject'] = study_combined_df['Study_Subject']
        study_combined_df['Study_Name'] = study_name_combined
        study_combined_df['Study_Subject'] = study_combined_df['Study_Name'] + '_____' + study_combined_df['Subject']
        study_combined_df = generate_ecdf_arousal_cols(study_combined_df, study_comb_signals)
        single_sec_df = pd.concat([single_sec_df, study_combined_df])

        aggr_df = get_non_overlapping_aggregrated_df(study_combined_df)
        aggr_df['Study_Name'] = study_name_combined
        aggr_df = get_study_subject_name(aggr_df)
        aggr_df = rename_cols(aggr_df)
        window_sec_df = pd.concat([window_sec_df, aggr_df])


    single_sec_df.to_csv(all_studies_data_dir + 'data_2_5_study_combined.csv') 
    window_sec_df.to_csv(all_studies_data_dir + 'data_3_study_combined____v0_delete_it.csv') # Need to clean the deadline study data

    total_time = time.time() - start_time
    print("--- Total time for combining studies %s mins (%s hours) ---" % (total_time//60, total_time//3600))
    #############################################################################################

In [221]:
if not combine_studies:
    window_sec_df = pd.read_csv(all_studies_data_dir + 'data_3_study_combined____v0_delete_it.csv')

In [222]:
# classification_df = window_sec_df.copy()
window_sec_df.columns

Index(['Unnamed: 0', 'Subject', 'Age', 'NASA_Effort', 'NASA_Frustration',
       'NASA_Mental', 'NASA_Performance', 'NASA_Physical', 'NASA_Temporal',
       'NASA_Total', 'Gender', 'Gender_Male', 'Perinasal_Mean_NN',
       'Perinasal_Median_NN', 'Perinasal_SD_NN', 'Perinasal_SS_NN',
       'Hr_Mean_NN', 'Hr_Median_NN', 'Hr_SD_NN', 'Hr_SS_NN', 'Br_Mean_NN',
       'Br_Median_NN', 'Br_SD_NN', 'Br_SS_NN', 'Perinasal_Mean',
       'Perinasal_Median', 'Perinasal_SD', 'Perinasal_SS', 'HR_Mean',
       'HR_Median', 'HR_SD', 'HR_SS', 'BR_Mean', 'BR_Median', 'BR_SD', 'BR_SS',
       'PP_Arousal_Mode', 'HR_Arousal_Mode', 'BR_Arousal_Mode',
       'PP_HR_Arousal_Mode', 'PP_BR_Arousal_Mode', 'HR_BR_Arousal_Mode',
       'PP_HR_BR_Arousal_Mode', 'Study_Name', 'Study_Subject', 'Treatment',
       'PP_RB_Mean', 'HR_RB_Mean', 'BR_RB_Mean', 'PP_Arousal_Mode_Hist',
       'HR_Arousal_Mode_Hist', 'BR_Arousal_Mode_Hist',
       'PP_HR_Arousal_Mode_Hist', 'PP_BR_Arousal_Mode_Hist',
       'HR_BR_Arousal_M

In [223]:
window_sec_df = window_sec_df[['Study_Name', 
                                       'Subject', 
                                       'Treatment',
                                       
                                       'Age', 
                                       'Gender', 
                                       
                                       'NASA_Physical',
                                       'NASA_Effort', 
                                       'NASA_Frustration',
                                       'NASA_Temporal',
                                       'NASA_Performance',
                                       ### 'NASA_Mental',
                                       ### 'NASA_Total',
                                       
                                       'Perinasal_Mean',
                                       'Perinasal_Median', 
                                       'Perinasal_SD', 
                                       'Perinasal_SS',
                                       
                                       'HR_Mean', 
                                       'HR_Median', 
                                       'HR_SD', 
                                       'HR_SS', 
                                       
                                       'BR_Mean',
                                       'BR_Median', 
                                       'BR_SD', 
                                       'BR_SS',
                                       
                                       'PP_Arousal_Mode', 
                                       'HR_Arousal_Mode', 
                                       'BR_Arousal_Mode',
                                       'PP_HR_Arousal_Mode', 
                                       'PP_BR_Arousal_Mode', 
                                       'HR_BR_Arousal_Mode',
                                       'PP_HR_BR_Arousal_Mode'
                                      ]]

window_sec_df.columns

Index(['Study_Name', 'Subject', 'Treatment', 'Age', 'Gender', 'NASA_Physical',
       'NASA_Effort', 'NASA_Frustration', 'NASA_Temporal', 'NASA_Performance',
       'Perinasal_Mean', 'Perinasal_Median', 'Perinasal_SD', 'Perinasal_SS',
       'HR_Mean', 'HR_Median', 'HR_SD', 'HR_SS', 'BR_Mean', 'BR_Median',
       'BR_SD', 'BR_SS', 'PP_Arousal_Mode', 'HR_Arousal_Mode',
       'BR_Arousal_Mode', 'PP_HR_Arousal_Mode', 'PP_BR_Arousal_Mode',
       'HR_BR_Arousal_Mode', 'PP_HR_BR_Arousal_Mode'],
      dtype='object')

In [224]:
ground_truth_cols = ['PP_Arousal_Mode', 'HR_Arousal_Mode', 'BR_Arousal_Mode', 'PP_HR_Arousal_Mode', 'PP_BR_Arousal_Mode', 'HR_BR_Arousal_Mode', 'PP_HR_BR_Arousal_Mode']
for ground_truth_col in ['PP_Arousal_Mode']:
    window_sec_df[ground_truth_col] = window_sec_df.apply(lambda x: change_ground_truth_labels(x[ground_truth_col]), axis=1)

window_sec_df.head()

,Study_Name,Subject,Treatment,Age,Gender,NASA_Physical,NASA_Effort,NASA_Frustration,NASA_Temporal,NASA_Performance,Perinasal_Mean,Perinasal_Median,Perinasal_SD,Perinasal_SS,HR_Mean,HR_Median,HR_SD,HR_SS,BR_Mean,BR_Median,BR_SD,BR_SS,PP_Arousal_Mode,HR_Arousal_Mode,BR_Arousal_Mode,PP_HR_Arousal_Mode,PP_BR_Arousal_Mode,HR_BR_Arousal_Mode,PP_HR_BR_Arousal_Mode
0,sim1,2,RD,23.0,1,12.0,17.0,11.0,11.0,8.0,0.020212,0.029017,0.022537,0.008657,3.708182,3.668182,1.484887,157.350124,-5.414682,-5.416682,0.162809,293.426352,neutral,stressed,relaxed,neutral,relaxed,neutral,neutral
1,sim1,2,RD,23.0,1,12.0,17.0,11.0,11.0,8.0,-0.056016,-0.059782,0.010595,0.032388,4.568182,3.968182,1.366260,225.482851,-4.462682,-4.366682,0.352742,200.275130,non-arousal,stressed,relaxed,neutral,relaxed,neutral,neutral
2,sim1,2,RD,23.0,1,12.0,17.0,11.0,11.0,8.0,-0.047614,-0.045814,0.007855,0.023226,3.448182,3.668182,1.347261,135.235579,-3.358682,-3.366682,0.355928,113.947596,non-arousal,stressed,relaxed,neutral,relaxed,neutral,neutral
3,sim1,2,RD,23.0,1,12.0,17.0,11.0,11.0,8.0,-0.121872,-0.129739,0.022728,0.153176,0.008182,0.168182,1.188089,12.704669,-2.302682,-2.386682,0.319138,53.940076,non-arousal,neutral,relaxed,relaxed,relaxed,neutral,relaxed
4,sim1,2,RD,23.0,1,12.0,17.0,11.0,11.0,8.0,-0.157789,-0.160660,0.008453,0.249618,0.068182,0.968182,1.708150,26.306488,-0.978682,-0.916682,0.339830,10.617541,non-arousal,neutral,neutral,neutral,relaxed,neutral,neutral


----
----
# !!!!!! Deadline Issues !!!!!!! 
#### 1. Replace all BR Realted features NA
#### 2. Remove the Treatments other than Corresponding_Day_RB
#### 3. Replace the Treatments as Day1_WS, Day1_WS etc.
----
----
----
----
----
# !!!!!! Check Other Things !!!!!!! 
#### 1. Check unique values for each categorical columns
#### 2. Check range for each numeric columns
----
----

# Do all data cleaning before exporting and scaling data_3_study_combined.csv
# Only column selection and renaming for the OSf. Nothing else!!!



In [226]:
window_sec_df[window_sec_df.Study_Name=='deadline_study']

,Study_Name,Subject,Treatment,Age,Gender,NASA_Physical,NASA_Effort,NASA_Frustration,NASA_Temporal,NASA_Performance,Perinasal_Mean,Perinasal_Median,Perinasal_SD,Perinasal_SS,HR_Mean,HR_Median,HR_SD,HR_SS,BR_Mean,BR_Median,BR_SD,BR_SS,PP_Arousal_Mode,HR_Arousal_Mode,BR_Arousal_Mode,PP_HR_Arousal_Mode,PP_BR_Arousal_Mode,HR_BR_Arousal_Mode,PP_HR_BR_Arousal_Mode
22112,deadline_study,5,Corresponding_Day_RB,35.0,0,5.0,15.0,5.0,12.0,12.0,0.346319,0.361038,0.055406,1.227000,-16.791476,-16.791476,0.001579,2819.536619,-16.791476,-16.791476,0.001579,2819.536619,arousal,relaxed,relaxed,neutral,neutral,relaxed,relaxed
22113,deadline_study,5,Corresponding_Day_RB,35.0,0,5.0,15.0,5.0,12.0,12.0,0.346319,0.361038,0.055406,1.227000,-16.791476,-16.791476,0.001579,2819.536619,-16.791476,-16.791476,0.001579,2819.536619,arousal,relaxed,relaxed,neutral,neutral,relaxed,relaxed
22114,deadline_study,5,Corresponding_Day_RB,35.0,0,5.0,15.0,5.0,12.0,12.0,0.346319,0.361038,0.055406,1.227000,-16.791476,-16.791476,0.001579,2819.536619,-16.791476,-16.791476,0.001579,2819.536619,arousal,relaxed,relaxed,neutral,neutral,relaxed,relaxed
22115,deadline_study,5,Day3_Day4_Mean,35.0,0,5.0,15.0,5.0,12.0,12.0,0.346319,0.361038,0.055406,1.227000,-16.791476,-16.791476,0.001579,2819.536619,-16.791476,-16.791476,0.001579,2819.536619,arousal,relaxed,relaxed,neutral,neutral,relaxed,relaxed
22116,deadline_study,5,Day3_Day4_Min,35.0,0,5.0,15.0,5.0,12.0,12.0,0.346319,0.361038,0.055406,1.227000,-16.791476,-16.791476,0.001579,2819.536619,-16.791476,-16.791476,0.001579,2819.536619,arousal,relaxed,relaxed,neutral,neutral,relaxed,relaxed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195490,deadline_study,19,Corresponding_Day_RB,52.0,0,10.0,5.0,2.0,10.0,18.0,1.198315,1.200738,0.045036,14.377842,6.781481,6.782414,0.008631,459.885460,6.781481,6.782414,0.008631,459.885460,arousal,stressed,stressed,stressed,stressed,stressed,stressed
195491,deadline_study,19,Day3_Day4_Mean,52.0,0,10.0,5.0,2.0,10.0,18.0,1.198315,1.200738,0.045036,14.377842,6.781481,6.782414,0.008631,459.885460,6.781481,6.782414,0.008631,459.885460,arousal,stressed,stressed,stressed,stressed,stressed,stressed
195492,deadline_study,19,Day3_Day4_Min,52.0,0,10.0,5.0,2.0,10.0,18.0,1.198315,1.200738,0.045036,14.377842,6.781481,6.782414,0.008631,459.885460,6.781481,6.782414,0.008631,459.885460,arousal,stressed,stressed,stressed,stressed,stressed,stressed
195493,deadline_study,19,Four_Day_Min,52.0,0,10.0,5.0,2.0,10.0,18.0,1.198315,1.200738,0.045036,14.377842,6.781481,6.782414,0.008631,459.885460,6.781481,6.782414,0.008631,459.885460,arousal,stressed,stressed,stressed,stressed,stressed,stressed


In [227]:
window_sec_df = window_sec_df[window_sec_df.Study_Name!='deadline_study']
window_sec_df[window_sec_df.Study_Name=='deadline_study']

,Study_Name,Subject,Treatment,Age,Gender,NASA_Physical,NASA_Effort,NASA_Frustration,NASA_Temporal,NASA_Performance,Perinasal_Mean,Perinasal_Median,Perinasal_SD,Perinasal_SS,HR_Mean,HR_Median,HR_SD,HR_SS,BR_Mean,BR_Median,BR_SD,BR_SS,PP_Arousal_Mode,HR_Arousal_Mode,BR_Arousal_Mode,PP_HR_Arousal_Mode,PP_BR_Arousal_Mode,HR_BR_Arousal_Mode,PP_HR_BR_Arousal_Mode


In [228]:
window_sec_df.to_csv(all_studies_data_dir + 'data_3_study_combined.csv') 
get_scaled_df(files=['data_3_study_combined.csv'])

data_3_study_combined.csv sim1


KeyError: "None of [Index(['PP_Mean'], dtype='object')] are in the [columns]"

----
# From here...do only column selection & renaming
----

In [ ]:
classification_df = pd.read_csv(all_studies_data_dir + 'data_3_study_combined.csv')
classification_df

In [ ]:
classification_df = rename_cols(classification_df)

classification_df.rename({
                        'Study_Name': 'Study',
                        'Subject': 'Participant_ID',
    
                        'PP_Arousal_Mode': 'PP_Ground_Truth',
                        # 'HR_Arousal_Mode': 'HR_Ground_Truth', 
                        # 'BR_Arousal_Mode': 'BR_Ground_Truth',
                        # 'PP_HR_Arousal_Mode', 
                        # 'PP_BR_Arousal_Mode', 
                        # 'HR_BR_Arousal_Mode',
                        # 'PP_HR_BR_Arousal_Mode'
                         }, 
                         axis=1, inplace=True)

classification_df.columns

In [ ]:
classification_df = classification_df[['Study', 
                                       'Participant_ID', 
                                       'Treatment',
                                       
                                       'Age', 
                                       'Gender', 
                                       
                                       'PP_Mean',
                                       'PP_SD',
                                       
                                       'PP_Ground_Truth']]


In [ ]:
classification_df.head()

In [ ]:
classification_df.Study.unique()

In [ ]:
classification_df.to_csv(data_dir + 'classification_df.csv') 


In [ ]:
# 'NASA_Physical',
# 'NASA_Effort', 
# 'NASA_Frustration',
# 'NASA_Temporal',
# 'NASA_Performance',
### 'NASA_Mental',
### 'NASA_Total',

# 'Perinasal_Mean': 'Perinasal_Mean',
# 'Perinasal_SD': 'Perinasal_SD', 
### 'Perinasal_Median': 'Perinasal_Median', 
### 'Perinasal_SS': 'Perinasal_SS',

# 'Hr_Mean': 'HR_Mean', 
# 'Hr_SD': 'HR_SD', 
### 'Hr_Median',
### 'Hr_SS', 

#####################
# 'Br_Mean': 'BR_Mean',
# 'Br_SD': 'BR_SD',
### 'Br_Median', 
### 'Br_SS'
#####################